In [48]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
import keras.backend as K

Se carga el dataset

In [57]:
print("Se está cargando el dataset, aguarde un momento...")
filePath = "C:/Users/raul.blanco/Documents/4 - Personales/UBA/Repositorios/IA/Trabajo-Final/solar_wind.csv"
dataset = pd.read_csv(filepath_or_buffer=filePath, header=0, sep=",")
dataset = dataset.dropna()
print("Dataset cargado correctamente.")

Se está cargando el dataset, aguarde un momento...
Dataset cargado correctamente.


Se realiza One Hot Encodding

In [50]:
#dataset_org = dataset
#dataset = pd.get_dummies(dataset, columns=['source', 'period'], prefix=['source', 'period'])

Se divide el dataset en train y test

In [58]:
train_size = int(len(dataset) * 0.8)
train_data = dataset[:train_size]
test_data = dataset[train_size:]

In [59]:
train_data.set_index(['timedelta'])
test_data.set_index(['timedelta'])

,period,bx_gse,by_gse,bz_gse,theta_gse,phi_gse,bx_gsm,by_gsm,bz_gsm,theta_gsm,phi_gsm,bt,density,speed,temperature,source
timedelta,,,,,,,,,,,,,,,,
1233 days 22:20:00,train_c,1.17,-2.54,0.65,13.16,294.66,1.16,-2.33,1.21,24.92,296.50,2.87,4.71,353.47,33302.0,ac
1233 days 22:21:00,train_c,1.13,-2.66,-0.17,-3.45,292.94,1.12,-2.63,0.43,8.54,293.04,2.89,5.12,349.25,41200.0,ac
1233 days 22:22:00,train_c,0.63,-2.98,-0.21,-3.86,281.96,0.63,-2.95,0.47,8.88,281.98,3.05,3.55,351.36,34041.0,ac
1233 days 22:23:00,train_c,1.76,-2.01,0.16,3.42,311.17,1.75,-1.92,0.61,13.15,312.32,2.67,4.81,353.56,25111.0,ac
1233 days 22:24:00,train_c,1.61,-2.20,0.16,3.31,306.17,1.60,-2.11,0.65,13.76,307.25,2.73,4.42,348.54,57336.0,ac
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435 days 23:55:00,train_c,-1.18,2.00,-3.92,-59.28,120.51,-1.17,2.98,-3.23,-45.24,111.48,4.56,4.64,348.73,23368.0,ac
2435 days 23:56:00,train_c,-1.34,2.12,-3.83,-56.78,122.23,-1.33,3.08,-3.12,-42.95,113.45,4.58,4.27,346.36,26497.0,ac
2435 days 23:57:00,train_c,-1.62,2.33,-3.63,-51.98,124.76,-1.61,3.22,-2.87,-38.57,116.58,4.61,3.95,344.12,27050.0,ac


Se introduce las anomalías en los dataset de entrenamiento y de testeo

In [60]:
feature = 'bt'
anomaly_percentage_train = 0.2
sample_indices_train = train_data.sample(frac=anomaly_percentage_train, random_state=42).index 
num_anomalies_train = int(len(train_data) * anomaly_percentage_train)+1
#train_data.loc[sample_indices_train, feature] = np.random.normal(loc=40, scale=1.5, size=len(sample_indices_train))
train_data.loc[sample_indices_train, feature] = np.random.uniform(low=0, high=100, size=num_anomalies_train)

In [61]:
anomaly_percentage_test = 0.01
sample_indices_test = test_data.sample(frac=anomaly_percentage_test, random_state=42).index
num_anomalies_test = int(len(test_data) * anomaly_percentage_test)+1
#test_data.loc[sample_indices_test, feature] = np.random.normal(loc=40, scale=1.5, size=len(sample_indices_test))
test_data.loc[sample_indices_test, feature] = np.random.uniform(low=0, high=100, size=len(sample_indices_test))

Etiqueto los registros que tienen anomalías

In [62]:
train_data['class'] = 0
train_data.loc[sample_indices_train, 'class'] = 1
test_data['class'] = 0
test_data.loc[sample_indices_test, 'class'] = 1

C:\Users\raul.blanco\AppData\Local\Temp\ipykernel_25268\893168558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['class'] = 0
C:\Users\raul.blanco\AppData\Local\Temp\ipykernel_25268\893168558.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['class'] = 0


Guardo los datasets

In [63]:
train_data.to_csv('train_data_u.csv')
print("Se guardó dataset de train con el nombre train_data_u.csv")
# guardo el dataset test
test_data.to_csv('test_data_u.csv')
print("Se guardó dataset de test con el nombre test_data_u.csv")

Se guardó dataset de train con el nombre train_data_u.csv
Se guardó dataset de test con el nombre test_data_u.csv
